In [ ]:
# Import initial dependences
import pandas as pd

In [ ]:
# Load data
myopia_df = pd.read_csv("Resources/myopia.csv")

In [ ]:
# View data
myopia_df.head()

In [ ]:
# Remove MYOPIC classifier since we're using unsupervised machine learning
new_myopia_df = myopia_df.drop("MYOPIC", axis='columns')

In [ ]:
# View data
new_myopia_df.head()